In [ ]:
!pip install pyserial

In [ ]:
!conda install -c anaconda pandas

In [ ]:
import serial
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import time
import struct

In [ ]:
class TactileSensor:
    PC_MEASURE_REQUEST = 0       
    sensorXNum=0
    sensorYNum=0
    thermalNum=0
    fingerNum=0
    pressureDistribution = 0
    
    def __init__(self, param = (10, 10, 2, 3)):  
        self.setParam(param)
    
    def setParam(self, param):  
        self.sensorXNum = param[0]  
        self.sensorYNum = param[1]  
        self.thermalNum = param[2]
        self.fingerNum = param[3]
        
    def serialInit(self, port = 'COM9', baudrate = 921600, request = 0xFE):
        self.ser = serial.Serial()
        self.ser.baudrate = baudrate
        self.ser.port = port  
        self.PC_MEASURE_REQUEST = request
        
    def firstRequest(self):
        self.serialInit()
        data = bytearray([self.PC_MEASURE_REQUEST])
        self.ser.open()
        self.ser.write(data)
        time.sleep(1)
        self.ser.flushInput()
        self.ser.close()
        self.ReadData()
            
    def ReadData(self):
        pressureDistribution = np.zeros([self.fingerNum, self.sensorXNum, self.sensorYNum], dtype = int)
        finger = self.fingerNum
        x = self.sensorXNum
        y = self.sensorYNum
        packing = 0
        data = bytearray([self.PC_MEASURE_REQUEST])
    
        self.ser.open()
        self.ser.write(data)
        time.sleep(1)    
        for finger in range(self.fingerNum):
            for x in range(self.sensorXNum):
                for y in range(self.sensorYNum):
                    if(packing == 0):
                        rcv = self.ser.read()                             #AAAAAAAA
                        rcv = struct.unpack('B',rcv[:1])
                        pressureDistribution[finger][x][y]= rcv[0]<<4     #AAAAAAAA0000
                        rcv = self.ser.read()                             #        AAAABBBB
                        rcv = struct.unpack('B',rcv[:1])
                        pressureDistribution[finger][x][y] |= (rcv[0]>>4) #AAAAAAAAAAAA
                    else:
                        pressureDistribution[finger][x][y] = (rcv[0] & 0x0F)<<8       #BBBB00000000
                        rcv = self.ser.read()                                         #    BBBBBBBB
                        rcv = struct.unpack('B',rcv[:1])
                        pressureDistribution[finger][x][y] |= rcv[0]                  #BBBBBBBBBBBB
        
                    packing = (packing+1)%2 #0,1,0,1,0,1,...   
            for x in range(self.thermalNum):            
                if(packing == 0):
                    rcv = self.ser.read()
                    rcv = self.ser.read()
                else:
                    rcv = self.ser.read()   
                packing = (packing+1)%2 
        self.ser.flushInput()
        self.ser.close()
        self.pressureDistribution = pressureDistribution
        self.Visualisation()
        
    def Visualisation(self): 
        presDis = self.pressureDistribution
        shape = presDis.shape
        xSP = 0
        ySP = 0
        step = 20
    
        plt.axes()
        for finger in range(shape[0]):
            for x in range(shape[1]):
                for y in range(shape[2]):
                    rectangle = plt.Rectangle((xSP+step*(x), ySP+step*(y)), step, step, 
                                              fc=(0.1, presDis[finger][x][y]/4096, 0.1), ec="red")
                    plt.gca().add_patch(rectangle)

            xSP += step*11
        plt.axis('scaled')
        plt.show()

In [ ]:
sensor = TactileSensor()
sensor.firstRequest()

In [ ]:
sensor.ReadData()

In [ ]:
from IPython.display import clear_output

In [ ]:
while True:
    sensor.ReadData()
    time.sleep(0.01)
    clear_output(wait=True)
    